In [ ]:
%pip install kagglehub


DOWNLOAD SYNTHEA AND DO ALL THE ANNOYING STEPS AGAIN

In [60]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uom190346a/disease-symptoms-and-patient-profile-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Tanis\.cache\kagglehub\datasets\uom190346a\disease-symptoms-and-patient-profile-dataset\versions\2


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Symptom_and_patient_profile_dataset.csv')
df.head()

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [3]:
#replacing string values with a numerical input
exclude = ['Age', 'Disease']
X = df.copy() #copy to not modify the original dataframe
X = X.dropna()

X = df.drop('Disease', axis=1)
y = df['Disease']

X

,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...
344,Yes,No,Yes,No,80,Female,High,High,Positive
345,Yes,No,Yes,No,85,Male,High,High,Positive
346,Yes,No,Yes,No,85,Male,High,High,Positive
347,Yes,No,Yes,No,90,Female,High,High,Positive


There are a lot of labels with few samples. This could impact the model's performance. There are also labels that dominate the data which could cause bias.

In [4]:
from collections import Counter
 # Need to handle dominant classes and remove some minority classes
print(Counter(y))

Counter({'Asthma': 23, 'Stroke': 16, 'Osteoporosis': 14, 'Diabetes': 10, 'Migraine': 10, 'Hypertension': 10, 'Influenza': 8, 'Bronchitis': 8, 'Pneumonia': 8, 'Hyperthyroidism': 7, 'Hypothyroidism': 7, 'Common Cold': 6, 'Eczema': 6, 'Allergic Rhinitis': 6, 'Anxiety Disorders': 6, 'Gastroenteritis': 6, 'Rheumatoid Arthritis': 6, 'Depression': 6, 'Liver Cancer': 6, 'Kidney Cancer': 6, 'Osteoarthritis': 6, 'Pancreatitis': 5, 'Urinary Tract Infection': 5, 'Ulcerative Colitis': 5, 'Multiple Sclerosis': 5, 'Psoriasis': 5, 'Liver Disease': 5, 'Kidney Disease': 5, 'Coronary Artery Disease': 5, "Crohn's Disease": 5, "Parkinson's Disease": 5, "Alzheimer's Disease": 5, 'Chronic Obstructive Pulmonary Disease (COPD)': 4, 'Hepatitis': 3, 'Urinary Tract Infection (UTI)': 3, 'Tuberculosis': 3, 'Lung Cancer': 3, 'Dengue Fever': 2, 'Rubella': 2, 'Malaria': 2, 'Klinefelter Syndrome': 2, 'Rabies': 2, 'Chickenpox': 2, 'Hemophilia': 2, 'Turner Syndrome': 2, 'Zika Virus': 2, 'Tetanus': 2, 'Cholera': 2, 'Ebola

In [5]:
len(y)

349

In [1]:
# need to undersample asthma, stroke, osteoporosis. perhaps oversample classes with a 2-4

In [81]:
counts = y.value_counts()
labels = counts[counts > 6].index # Removing value counts of 2
mask = y.isin(labels)

X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

In [32]:
df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
45,Hypothyroidism,No,Yes,No,No,27,Male,Normal,Low,Positive
46,Hypothyroidism,Yes,No,Yes,No,22,Female,Normal,Normal,Negative
47,Hypothyroidism,No,Yes,No,Yes,55,Male,High,Low,Positive
48,Hypothyroidism,No,Yes,No,Yes,19,Male,Low,Low,Positive


In [69]:
y.value_counts()

Disease
Asthma             23
Influenza          18
Bronchitis         18
Pneumonia          18
Hyperthyroidism    17
Hypothyroidism     17
Stroke             16
Osteoporosis       14
Diabetes           10
Migraine           10
Hypertension       10
Name: count, dtype: int64

YES — **you should absolutely do the resampling (like random undersampling or SMOTE) 🔴ONLY ON THE TRAINING SET🔴 — not the whole dataset. Otherwise, you're leaking artificial/manipulated data into your test set, and that ruins everything.

In [ ]:
# Random under sampling
asthma = y[y == "Asthma"]
stroke = y[y == "Stroke"]
osteoporosis = y[y == "Osteoporosis"]

# Medium-sized class to match the majority classes with
kidney_cancer = y[y == "Bronchitis"]

asthma = asthma.sample(n=len(kidney_cancer))
#stroke = stroke.sample(n=len(kidney_cancer))
#osteoporosis = osteoporosis.sample(n=len(kidney_cancer))

# Filtering out unwanted labels
y_remaining = y[~y.isin(["Asthma", "Stroke", "Osteoporosis"])]

# Appending the remaining labels with the new labels
#you dropped stuff from y so the indexes in y arent the same in x
#so you have to do reset_index(drop=True)
y = pd.concat([y_remaining, asthma, stroke, osteoporosis], axis=0).reset_index(drop=True)
X = X.loc[y.index].reset_index(drop=True)

y

ValueError: Cannot take a larger sample than population when 'replace=False'

In [82]:
X = X.loc[y.index].reset_index(drop=True)

In [83]:
X

,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
2,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
3,Yes,Yes,Yes,Yes,25,Female,Normal,Normal,Positive
4,Yes,Yes,Yes,Yes,25,Female,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...
166,No,No,No,Yes,63,Female,Normal,Low,Positive
167,Yes,Yes,No,Yes,70,Female,High,Low,Negative
168,No,No,No,Yes,40,Male,Normal,Low,Negative
169,No,No,No,No,30,Female,Low,High,Positive


In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, stratify=y, test_size=0.2)

In [73]:
from imblearn.over_sampling import SMOTEN

smoten = SMOTEN(random_state=42)
X_train, y_train = smoten.fit_resample(X_train, y_train)

In [87]:
print(Counter(y_train))

Counter({'Asthma': 18, 'Bronchitis': 14, 'Hyperthyroidism': 14, 'Hypothyroidism': 14, 'Pneumonia': 14, 'Influenza': 14, 'Stroke': 13, 'Osteoporosis': 11, 'Diabetes': 8, 'Hypertension': 8, 'Migraine': 8})


In [85]:
models = {
    'decision_tree' : DecisionTreeClassifier(class_weight='balanced'),
    'random_forest' : RandomForestClassifier(class_weight='balanced'),
    'svc' : SVC(class_weight='balanced'),
    'cat_boost' : CatBoostClassifier(verbose=0),
    'logistic_regression' : LogisticRegression(class_weight='balanced'),
    'k_neighbors' : KNeighborsClassifier(),
    'ada_boost' : AdaBoostClassifier(),
    #'xgb' : XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [86]:
def unknown_encoding(labels, label_encoder):
    known_classes = set(label_encoder.classes_)
    encoded = []

    for label in labels:
        if label in known_classes:
            encoded.append(label_encoder.transform([label])[0])
        else:
            encoded.append(-1)
    return np.array(encoded)

In [88]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = unknown_encoding(y_test, label_encoder)

one_hot_encoder.fit(X_train)
X_train_encoded = one_hot_encoder.transform(X_train)
X_test_encoded = one_hot_encoder.transform(X_test)

In [89]:
Counter(y_train)

Counter({'Asthma': 18,
         'Bronchitis': 14,
         'Hyperthyroidism': 14,
         'Hypothyroidism': 14,
         'Pneumonia': 14,
         'Influenza': 14,
         'Stroke': 13,
         'Osteoporosis': 11,
         'Diabetes': 8,
         'Hypertension': 8,
         'Migraine': 8})

In [57]:
X_train_encoded

array([[0., 1., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [24]:
Counter(y_train_encoded)

Counter({np.int64(9): 18,
         np.int64(3): 18,
         np.int64(7): 18,
         np.int64(6): 18,
         np.int64(0): 18,
         np.int64(1): 18,
         np.int64(2): 18,
         np.int64(10): 18,
         np.int64(8): 18,
         np.int64(5): 18,
         np.int64(4): 18})

In [90]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def print_metrics(y_test, predicted, model):
    print(f"{model}")
    print(f"Accuracy: {accuracy_score(y_test, predicted):.3f}")
    print(f"Precision Score: {precision_score(y_test, predicted, average='macro'):.3f}")
    print(f"Recall Score: {recall_score(y_test, predicted, average='macro'):.3f}")
    print(f"F1 Score: {f1_score(y_test, predicted, average='macro'):.3f}")
    print("Confusion Matrix")
    print(confusion_matrix(y_test, predicted))
    print()

In [91]:
for name, model in models.items():
    model.fit(X_train_encoded, y_train_encoded)
    predicted = model.predict(X_test_encoded)
    print_metrics(y_test_encoded, predicted, name)

decision_tree
Accuracy: 0.400
Precision Score: 0.500
Recall Score: 0.388
F1 Score: 0.385
Confusion Matrix
[[3 0 0 0 1 1 0 0 0 0 0]
 [0 2 0 1 0 0 1 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 1 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 1]
 [0 0 0 1 0 0 1 1 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 1 0 0]
 [1 0 1 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 3]]

random_forest
Accuracy: 0.343
Precision Score: 0.263
Recall Score: 0.338
F1 Score: 0.287
Confusion Matrix
[[4 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 1 1 0]
 [0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 1]
 [0 0 0 0 1 2 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 0 1 0 1 0 0]
 [2 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 3]]

svc
Accuracy: 0.200
Precision Score: 0.242
Recall Score: 0.223
F1 Score: 0.210
Confusion Matrix
[[1 0 0 0 0 1 1 0 0 2 0]
 [1 0 0 0 1 0 0 0 0 2 0]
 [0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 2 0 0 0]
 [0 0 0 0 1 0

C:\Users\Tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


cat_boost
Accuracy: 0.343
Precision Score: 0.294
Recall Score: 0.338
F1 Score: 0.308
Confusion Matrix
[[4 0 0 0 0 1 0 0 0 0 0]
 [2 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 0 1 0 1 0 0]
 [2 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 3]]

logistic_regression
Accuracy: 0.171
Precision Score: 0.167
Recall Score: 0.150
F1 Score: 0.156
Confusion Matrix
[[2 0 0 0 0 1 1 0 0 1 0]
 [1 0 0 0 1 1 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 2 0 0 0]
 [0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 1 2 0 1 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 1 0 0]
 [3 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 2]]

k_neighbors
Accuracy: 0.229
Precision Score: 0.253
Recall Score: 0.229
F1 Score: 0.210
Confusion Matrix
[[3 0 0 0 0 1 0 0 0 1 0]
 [3 0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [2 0 0 1 0 0 0 0 0 0 0]
 [0

C:\Users\Tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati